# Import packages

In [47]:
import numpy as np

from collections import Counter

# Create a Sudoku class

In [48]:
class Sudoku():
  """ Describes a Sudoku puzzle

  Attributes:
  -----------

  sudo (np.array): 
    Integer Numpy array of shape (9,9) that holds the Sudoku's entries.
  
  Methods:
  --------

  print(): 
    Prints the Sudoku to stdout.

  possibilities(i: int, j: int) -> set: 
    Returns a set of integers holding all possible numbers for the field with 
    row index i and column index j.
  
  is_valid() -> bool:
    Checks if the Sudoku is valid.

  solve() -> bool:
    Tries to solve the Sudoku in-place and returns True if it was successful
    and False otherwise.
  
  solve_helper() -> bool:
    Auxiliary function to solve the Sudoku called by solve().
  """


  def __init__(self, A = np.zeros([9,9], int)):
    """ Initializes the Sudoku

    Args:
      A (np.array) = np.zeros([9,9], int): Integer Numpy array that holds
        the entries of the Sudoku
    """

    self.sudo = np.array(A, int)


  def print(self):
    """ Prints the Sudoku to stdout
    """

    for i in range(9):
      line = ""

      # Create the content of the new line
      for j in range(9):

        # Every third letter is "|"
        if not j % 3 and j != 0:
          line += "| "

        # If the entry is not 0 print it, otherwise put a whitespace
        if self.sudo[i,j] != 0:
          line += str(self.sudo[i,j]) + " "
        else:
          line += "  "

      # Every third row put a horizontal line  
      if not i % 3 and i != 0:
        print( "---------------------" )
      print(line)
  
  
  def possibilities(self, i, j):
    """ Returns a set of integers holding all possible numbers for the field 
    with row index i and column index j.

    Args:
      i (int): Row index
      j (int): Column index
    
    Returns:
      (set): a set of integers holding all possible numbers for the field 
        with row index i and column index j.
    """

    complement = []

    # Add all numbers in the same row to complement
    for k in range(9):
      x = self.sudo[i,k]
      if x != 0:
        complement += [x]

    # Add all numbers in the same column to complement
    for k in range(9):
      x = self.sudo[k,j]
      if x != 0:
        complement += [x]
    
    # Add all numbers in the same 3x3-square to complement
    a = i // 3
    b = j // 3
    for k in range(3):
      for l in range(3):
        x = self.sudo[3*a + k, 3*b + l]
        if x != 0:
          complement += [x]
    
    # Return possible numbers
    return set(range(1,10)) - set(complement)


  def is_valid(self):
    """ Checks if the Sudoku is valid.

    Returns:
      (bool): True if the Sudoku is valid, False otherwise.
    """

    # Check each row
    for i in range(9):

      # Count how often each number appears in row i
      c = Counter(self.sudo[i,:])

      # If a number appears more than once the Sudoku is invalid
      for k, v in c.items():

        # Note that 0 indicates an empty field
        if k != 0 and v > 1:
          return False

    for j in range(9):

      # Count how often each number appears in column j
      c = Counter(self.sudo[:,j])

      # If a number appears more than once the Sudoku is invalid
      for k, v in c.items():

        # Note that 0 indicates an empty field
        if k != 0 and v > 1:
          return False

    for i in range(3):
      for j in range(3):
        c = Counter()

        for k in range(3):
          c.update(self.sudo[3*i + k, 3*j : 3*j + 3])
        
        for k, v in c.items():

          # Note that 0 indicates an empty field
          if k != 0 and v > 1:
            return False

    return True

  
  def solve(self):
    """ Tries to solve the Sudoku in-place.

    Returns:
      (bool): True if it could solve the Sudoku, False otherwise.
    
    Raises:
      (ValueError): If the given Sudoku is not valid.
    """

    if self.is_valid():

      return self.solve_helper()
    else:

      raise ValueError('Sudoku is invalid. Invalid Sudokus cannot be solved.')

  
  def solve_helper(self):
    """ Tries to recursively solve the Sudoku in-place.

    Returns:
      (bool): True if it could solve the Sudoku, False otherwise.
    """

    # Solve the Sudoku recursively in-place:

    # i0, j0 is the position of the first 0 entry
    i0, j0 = None, None

    # Find the first 0 entry and store its position in i0, j0
    for i in range(9):
      for j in range(9):
        if self.sudo[i,j] == 0:
          i0 = i
          j0 = j
          break

    # If the Sudoku is completely filled in we are done
    if i0 is None:
      return True
    
    # Otherwise we consider all the possible numbers at i0, j0
    pos = self.possibilities(i0,j0)

    if pos:

      # If there are some possibilities...
      for x in pos:

        # ... iterate through them...
        self.sudo[i0,j0] = x

        # ... and see if they solve the Sudoku:
        if self.solve_helper():
          return True

      # If none of the possible entries at i0, j0 can solve the Sudoku
      # we clear the field...
      self.sudo[i0,j0] = 0

      # ... and return False
      return False

    # If there are no possible entries at i0, j0 we clear the field...
    self.sudo[i0,j0] = 0

    # ... and return False
    return False

# Test run with a valid Sudoku

In [49]:
A = [
     [0,6,0,0,1,0,8,7,0],
     [0,9,0,0,8,0,6,0,0],
     [0,0,0,6,2,0,0,0,0],
     [0,0,0,9,6,0,0,0,0],
     [0,0,5,0,0,0,0,0,2],
     [0,7,9,0,0,0,0,0,5],
     [0,0,0,0,0,4,0,0,0],
     [2,0,0,0,0,0,0,8,0],
     [8,0,0,0,0,0,0,5,1]
    ]
s = Sudoku(A)
s.print()

  6   |   1   | 8 7   
  9   |   8   | 6     
      | 6 2   |       
---------------------
      | 9 6   |       
    5 |       |     2 
  7 9 |       |     5 
---------------------
      |     4 |       
2     |       |   8   
8     |       |   5 1 


In [50]:
s.solve()
s.print()

5 6 2 | 4 1 3 | 8 7 9 
1 9 4 | 5 8 7 | 6 2 3 
7 3 8 | 6 2 9 | 5 1 4 
---------------------
3 2 1 | 9 6 5 | 7 4 8 
6 8 5 | 7 4 1 | 3 9 2 
4 7 9 | 2 3 8 | 1 6 5 
---------------------
9 1 7 | 8 5 4 | 2 3 6 
2 5 3 | 1 9 6 | 4 8 7 
8 4 6 | 3 7 2 | 9 5 1 


# Test run with an *invalid* Sudoku

In [51]:
A = np.zeros([9,9])
A[0,0] = 3
A[1,1] = 3
invalid_sudoku = Sudoku(A)
invalid_sudoku.print()
invalid_sudoku.solve()

3     |       |       
  3   |       |       
      |       |       
---------------------
      |       |       
      |       |       
      |       |       
---------------------
      |       |       
      |       |       
      |       |       


ValueError: ignored